# EmoAtlas - Starting guide
EmoAtlas is a Python library that checks against the input text, after having enriched it and structured as a semantic network, against the multilingual [NRC Lexicon](https://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm). The library is built upon the [Formamentis Networks](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0222870) from Stella et al. and the [PyPlutchik library](https://www.github.com/alfonsosemeraro/pyplutchik) (paper [here](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0256503)).

In particular, EmoAtlas offers two main functions, the first is to the first and most unique is to extract a Formamentis Network from the input text. The second is to plot the emotions associated with the text with the dyads Plutchik flower.

By combining these functions, the semantic and syntactic meaning of the text can be analysed in greater depth.


---

## Setup

### Installation

To install this library, you can run this command:

```
pip install -q git+https://github.com/alfonsosemeraro/emoatlas
```

The package "spaCY" will require a language "pipeline" to be installed. 

The choice of this pipeline will be determined by your needs and the language you are interested in.

```
English
python -m spacy download en_core_web_lg

Italian
python -m spacy download it_core_news_lg
```

This library uses Natural Language Toolkit (NLTK) as a core dependency. If this is the first time you're using NLTK, you need to download its data depending on the language you are interested in.

In [1]:
import nltk
nltk.download('wordnet') #English
# nltk.download('omw-1.4') #Italian


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ricca\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### Google Colab:
If you are using Google Colab, you can use these lines to quickly install a working version of this package.

In [2]:
# !pip install -q git+https://github.com/alfonsosemeraro/emoatlas
# !python -m spacy download en_core_web_lg
# import nltk
# nltk.download('wordnet')

---

## Using EmoScores

To start using the library, it is necessary to import the following libraries:

In [3]:
import matplotlib
from emoatlas import EmoScores
from collections import namedtuple #Used in this demo and to merge FMNTs

The EmoScores is the core of EmoAtlas. This package uses this class to understand how to associate emotion scores to each word. Many options are available ( different emotion lexicons, emotion models, lemmatization/stemming). ❗ **By Default, EmoAtlas uses the english language, the plutchik emotion model and implements lemmatization.** ❗

Stemming is the process of reducing words to their root form by removing suffixes. For example, the words "running" and "runs" would be reduced to the stem "run". 

**Lemmatization**, on the other hand, is the process of reducing words to their dictionary form, known as the lemma. For example, "am", "are", and "is" would all be lemmatized to "be".

The class EmoScores() has many options that we can use to personalize its usage, here's a short list of some of the most important attributes and methods:
- \*language\*: allows you to set which language you are going to analyze.
- .set_stemming_lemmatization(stem_or_lem='stemming'): that we cause to change the strategy to stemming.

All the other plots and analyses will be performed as EmoScores methods.

In [4]:
emos = EmoScores()

# If you are interested in Italian (requires to download the necessary nltk data):
# emos = EmoScores(language='italian')

---

# Creating FormaMentis Networks

Once we have used EmoScores 

In [5]:
sample_text = """In this text, we prefer dogs becase they are happy and dogs are positive.
Good dogs make good friends. Cats however are bad and have negative links. We dislike cats."""
fmnt = emos.formamentis_network(sample_text) 

In [6]:
print(fmnt)

FormamentisNetwork(edges=[('prefer', 'they'), ('text', 'we'), ('dog', 'they'), ('happy', 'they'), ('happy', 'prefer'), ('happy', 'we'), ('dog', 'prefer'), ('dog', 'positive'), ('dog', 'we'), ('positive', 'prefer'), ('positive', 'we'), ('dog', 'friend'), ('dog', 'good'), ('cat', 'link'), ('bad', 'link'), ('prefer', 'text'), ('dog', 'text'), ('prefer', 'we'), ('they', 'we'), ('dog', 'happy'), ('friend', 'good'), ('bad', 'cat'), ('link', 'negative'), ('cat', 'dislike'), ('dislike', 'we'), ('cat', 'we')], vertices=['cat', 'text', 'they', 'positive', 'we', 'link', 'bad', 'negative', 'good', 'dislike', 'happy', 'friend', 'prefer', 'dog'])
